In [36]:
from typing import Tuple

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from tick.hawkes import SimuHawkes, HawkesKernelSumExp, HawkesKernelExp
from tick.hawkes import HawkesExpKern

## Simulation

In [4]:
SEED = 1398
total_run_time = 100
time_step = 0.01
true_intensity = 0.5 # alpha
true_decay = 3 # beta
true_baseline = 1 # mu
number_of_nodes = 1  # D
hawkes = SimuHawkes(n_nodes=number_of_nodes, 
                    end_time=total_run_time, 
                    verbose=False, 
                    seed=SEED)
kernel = HawkesKernelExp(intensity=true_intensity, 
                            decay= true_decay)
hawkes.set_kernel(0, 0, kernel)
hawkes.set_baseline(0, true_baseline)
hawkes.track_intensity(time_step)
hawkes.simulate()
timestamps = hawkes.timestamps
intensity = hawkes.tracked_intensity
intensity_times = hawkes.intensity_tracked_times

## Fitting

In [5]:
decay_candidates = np.linspace(0, 30, 100)

In [6]:
decay_candidates

array([ 0.        ,  0.3030303 ,  0.60606061,  0.90909091,  1.21212121,
        1.51515152,  1.81818182,  2.12121212,  2.42424242,  2.72727273,
        3.03030303,  3.33333333,  3.63636364,  3.93939394,  4.24242424,
        4.54545455,  4.84848485,  5.15151515,  5.45454545,  5.75757576,
        6.06060606,  6.36363636,  6.66666667,  6.96969697,  7.27272727,
        7.57575758,  7.87878788,  8.18181818,  8.48484848,  8.78787879,
        9.09090909,  9.39393939,  9.6969697 , 10.        , 10.3030303 ,
       10.60606061, 10.90909091, 11.21212121, 11.51515152, 11.81818182,
       12.12121212, 12.42424242, 12.72727273, 13.03030303, 13.33333333,
       13.63636364, 13.93939394, 14.24242424, 14.54545455, 14.84848485,
       15.15151515, 15.45454545, 15.75757576, 16.06060606, 16.36363636,
       16.66666667, 16.96969697, 17.27272727, 17.57575758, 17.87878788,
       18.18181818, 18.48484848, 18.78787879, 19.09090909, 19.39393939,
       19.6969697 , 20.        , 20.3030303 , 20.60606061, 20.90

In [7]:
scores = []
hawkes_models = {}
for i, decay in enumerate(decay_candidates):
    hawkes_learner = HawkesExpKern(decay, 
                                   verbose=False, 
                                   max_iter=10000,
                                   tol=1e-10)
    hawkes_learner.fit(timestamps)
    hawkes_models[i] = hawkes_learner
    scores.append(hawkes_learner.score() )
    
    


Step equals 0... at 0


In [8]:
decay_scores_df = pd.DataFrame({'score': scores, 
                               'decay':decay_candidates}).sort_values(by = 'score', ascending = False)

In [9]:
best_hawkes_model_index = decay_scores_df.index[0]
best_hawkes_model = hawkes_models[best_hawkes_model_index]
beta = best_hawkes_model.decays
alpha = best_hawkes_model.adjacency[0][0]
baseline = best_hawkes_model.baseline[0]

In [14]:
learned_parameters  = baseline , alpha, beta

In [15]:
learned_parameters

(1.0766456880520174, 0.46659430920350237, 3.3333333333333335)

In [16]:
true_baseline, true_intensity, true_decay 


(1, 0.5, 3)

In [17]:
def conditional_intensity(params, events, t):
    baseline, alpha, beta = params
    intensity = baseline + np.sum(alpha*beta * np.exp(-beta * (t - events)))
    return intensity
# Predict the next arrival time
# current_time = events[-1]
# predicted_intensity = conditional_intensity(learned_params, events, current_time)
# next_arrival_time = current_time + np.random.exponential(1 / predicted_intensity)




In [18]:
timestamps[0][-2]

97.74902707032999

In [19]:
predicted_intensity=conditional_intensity(learned_parameters, timestamps[0], timestamps[0][-1])

In [20]:
next_arrival_time = timestamps[0][-1] + np.random.exponential(1 / predicted_intensity)

In [21]:
next_arrival_time

100.10642847814678

In [22]:
intensity

[array([1.        , 1.        , 1.        , ..., 2.10496554, 2.07230887,
        2.04061736])]

In [23]:
tracked_intensity, intensity_tracked_times = best_hawkes_model.estimated_intensity(timestamps, intensity_track_step=1)

In [24]:
intensity_tracked_times

array([ 0.26171969,  0.33306504,  1.        ,  1.13396494,  2.        ,
        3.        ,  3.36246705,  3.39104133,  4.        ,  4.20442785,
        5.        ,  6.        ,  6.56431294,  7.        ,  8.        ,
        8.22239123,  8.47910229,  8.5608451 ,  8.80829512,  9.        ,
        9.51074979, 10.        , 11.        , 12.        , 12.91835534,
       12.9793872 , 13.        , 13.1372456 , 13.77328057, 14.        ,
       14.10622039, 14.29056757, 14.30602899, 14.42221321, 14.42668694,
       14.59880966, 14.71865985, 15.        , 15.53903196, 15.69886662,
       15.70524077, 15.716086  , 16.        , 16.54016101, 17.        ,
       17.35684456, 18.        , 19.        , 19.61873656, 19.69200972,
       19.87800595, 20.        , 20.11658711, 20.58206516, 20.62337291,
       20.74143814, 21.        , 21.03131609, 21.15999083, 21.37426101,
       21.66128908, 21.68843724, 21.78395795, 21.84937938, 21.88533687,
       21.9700865 , 21.97387537, 21.99668448, 22.        , 22.07

In [25]:
tracked_intensity

[array([1.07664569, 2.30277201, 1.37779547, 1.26933092, 1.17410586,
        1.08012248, 1.0776843 , 2.49160058, 1.4668013 , 1.27402346,
        1.20023957, 1.08105478, 1.07731777, 1.44079894, 1.08963649,
        1.0828357 , 1.74026021, 2.76634162, 2.49895968, 2.64827185,
        1.36303921, 1.4371899 , 1.08950774, 1.07710453, 1.07666718,
        2.34567457, 3.71344695, 2.74540631, 1.46359155, 1.98886114,
        1.71686337, 2.26424492, 3.68177581, 3.90117343, 5.39165838,
        4.384064  , 4.33786109, 2.96225912, 1.38934288, 2.17311173,
        3.67267763, 5.08059788, 3.23443923, 1.43313518, 1.4894633 ,
        1.20229867, 1.27365938, 1.08367395, 1.07753927, 2.29561805,
        2.56907655, 3.10607621, 2.45257797, 1.69780623, 2.97315322,
        3.40544171, 2.71717857, 2.55456343, 3.05193115, 2.80511089,
        2.33806162, 3.64967226, 4.07922974, 4.74150594, 5.70718125,
        5.74014297, 7.21741042, 8.20927991, 9.66910594, 7.72856788,
        7.41905202, 3.15790851, 3.9169954 , 4.44

In [26]:
timestamps

[array([ 0.26171969,  0.33306504,  1.13396494,  3.36246705,  3.39104133,
         4.20442785,  6.56431294,  8.22239123,  8.47910229,  8.5608451 ,
         8.80829512,  9.51074979, 12.91835534, 12.9793872 , 13.1372456 ,
        13.77328057, 14.10622039, 14.29056757, 14.30602899, 14.42221321,
        14.42668694, 14.59880966, 14.71865985, 15.53903196, 15.69886662,
        15.70524077, 15.716086  , 16.54016101, 17.35684456, 19.61873656,
        19.69200972, 19.87800595, 20.11658711, 20.58206516, 20.62337291,
        20.74143814, 21.03131609, 21.15999083, 21.37426101, 21.66128908,
        21.68843724, 21.78395795, 21.84937938, 21.88533687, 21.9700865 ,
        21.97387537, 21.99668448, 22.07679377, 22.15412118, 22.55420099,
        22.62833571, 22.70859453, 22.89854451, 22.97865435, 23.43212217,
        23.62616571, 25.00303346, 26.92328088, 26.97909966, 27.06716701,
        27.86176803, 28.57450422, 29.21363031, 29.27725493, 30.17000552,
        30.21757082, 30.41137168, 31.57034694, 32.9

In [27]:
intensity_tracked_times

array([ 0.26171969,  0.33306504,  1.        ,  1.13396494,  2.        ,
        3.        ,  3.36246705,  3.39104133,  4.        ,  4.20442785,
        5.        ,  6.        ,  6.56431294,  7.        ,  8.        ,
        8.22239123,  8.47910229,  8.5608451 ,  8.80829512,  9.        ,
        9.51074979, 10.        , 11.        , 12.        , 12.91835534,
       12.9793872 , 13.        , 13.1372456 , 13.77328057, 14.        ,
       14.10622039, 14.29056757, 14.30602899, 14.42221321, 14.42668694,
       14.59880966, 14.71865985, 15.        , 15.53903196, 15.69886662,
       15.70524077, 15.716086  , 16.        , 16.54016101, 17.        ,
       17.35684456, 18.        , 19.        , 19.61873656, 19.69200972,
       19.87800595, 20.        , 20.11658711, 20.58206516, 20.62337291,
       20.74143814, 21.        , 21.03131609, 21.15999083, 21.37426101,
       21.66128908, 21.68843724, 21.78395795, 21.84937938, 21.88533687,
       21.9700865 , 21.97387537, 21.99668448, 22.        , 22.07

In [28]:
best_hawkes_simulator = SimuHawkes(n_nodes=number_of_nodes, 
                                    end_time=1, 
                                    verbose=False, 
                                    seed=SEED)
best_learned_kernel = HawkesKernelExp(intensity=best_hawkes_model.adjacency[0][0], 
                                        decay= best_hawkes_model.decays)
best_hawkes_simulator.set_kernel(0, 0, best_learned_kernel)
best_hawkes_simulator.set_baseline(0, best_hawkes_model.baseline[0])
best_hawkes_simulator.simulate()


In [29]:
predicted_time_step = best_hawkes_simulator.timestamps


In [30]:
predicted_time_step

[array([0.24308804, 0.3108563 ])]

In [31]:
best_hawkes_simulator.timestamps

[array([0.24308804, 0.3108563 ])]

In [32]:
intensity = best_hawkes_simulator.tracked_intensity
intensity_times = best_hawkes_simulator.intensity_tracked_times

ValueError: Intensity has not been tracked, you should call track_intensity before simulation

In [37]:
class UnivariateHawkes():
    """ """
    
    def __init__(self,
                baseline:float = None, 
                alpha: float = None,
                beta: float = None,
                total_run_time: int = 100,
                time_step: float = 0.01,
                max_iter: int = 10000,
                tol: float = 1e-10,
                min_beta: float = 0,
                max_beta: float = 30,
                verbose: bool = False,
                seed: int = 1500
                ):
        self.number_of_nodes = 1
        self.baseline = baseline
        self.alpha = alpha
        self.beta = beta
        self.total_run_time = total_run_time
        self.time_step = time_step
        self.max_iter = max_iter
        self.tol= tol 
        self.min_beta = min_beta
        self.max_beta = max_beta
        self.verbose = verbose
        self.seed = seed 
        
        
        
    def fit(self, events):
            
        beta_candidates = np.linspace(0, 30, 100)
        scores = []
        hawkes_models = {}
        for i, beta in enumerate(beta_candidates):
            hawkes_learner = HawkesExpKern(beta, 
                                           verbose=self.verbose, 
                                           max_iter=self.max_iter,
                                           tol=self.tol)
            hawkes_learner.fit(timestamps)
            hawkes_models[i] = hawkes_learner
            scores.append(hawkes_learner.score() )
    
        beta_scores_df = pd.DataFrame({'score': scores, 
                               'decay':beta_candidates}).sort_values(by = 'score', ascending = False)
        best_hawkes_model_index = beta_scores_df.index[0]
        best_hawkes_model = hawkes_models[best_hawkes_model_index]
        self.baseline = best_hawkes_model.baseline[0]
        self.alpha = best_hawkes_model.adjacency[0][0]
        self.beta = best_hawkes_model.decays
        
        
            
    
    def simulate(self)-> Tuple[np.ndarray, np.ndarray, np.ndarray]:
         
        hawkes = SimuHawkes(n_nodes=self.number_of_nodes, 
                            end_time=self.total_run_time, 
                            verbose=self.verbose, 
                            seed=self.seed)
        kernel = HawkesKernelExp(intensity=self.alpha, 
                                    decay= self.beta)
        hawkes.set_kernel(0, 0, kernel)
        hawkes.set_baseline(0, self.baseline)
        hawkes.track_intensity(self.time_step)
        hawkes.simulate()
        
        return hawkes.timestamps, hawkes.tracked_intensity, hawkes.intensity_tracked_times
        
        
    
    
    
    def get_conditional_intensity(self,
                                  events: np.ndarray, 
                                  current_step: float) -> float:

        return self.baseline + np.sum(self.alpha*self.beta * np.exp(-self.beta * (current_step - events)))            
    
    
    def get_next_arrival_time(self,
                                  events: np.ndarray, 
                                  current_step: float)-> float:
        
        predicted_intensity = self.get_conditional_intensity(events, current_step)
        
        return current_step + np.random.exponential(1 / predicted_intensity)
    

In [38]:
SEED = 1398
total_run_time = 100
time_step = 0.01
true_alpha = 0.5 # self-excitation intensity
true_beta = 3 # decay
true_baseline = 1 # mu


In [39]:
hawkes_model = UnivariateHawkes(baseline        = true_baseline, 
                                alpha           = true_alpha,
                                beta            = true_beta,
                                total_run_time  = total_run_time,
                                time_step       = time_step,
                                max_iter        = 10000,
                                tol             = 1e-10,
                                min_beta        = 0,
                                max_beta        = 30,
                                verbose         = False,
                                seed            = SEED)

In [40]:
simulated_info = hawkes_model.simulate()

In [41]:
simulated_events, simulated_tracked_intensity, simulated_intensity_tracked_times = simulated_info

In [42]:
hawkes_model.fit(simulated_events)

Step equals 0... at 0


In [45]:
true_baseline, true_alpha, true_beta

(1, 0.5, 3)

In [44]:
hawkes_model.baseline, hawkes_model.alpha, hawkes_model.beta

(1.0766456880520174, 0.46659430920350237, 3.3333333333333335)

In [50]:
hawkes_model.get_next_arrival_time(events=simulated_events[0], 
                                   current_step=simulated_events[0][-1])

100.28695364987065

In [ ]:
class UnivariateHawkes():
    """ """
    
    def __init__(self,
                baseline:float = None, 
                alpha: float = None,
                beta: float = None,
                total_run_time: int = 100,
                time_step: float = 0.01,
                max_iter: int = 10000,
                tol: float = 1e-10,
                min_beta: float = 0,
                max_beta: float = 30,
                verbose: bool = False,
                seed: int = 1500
                ):
        self.number_of_nodes = 1
        self.baseline = baseline
        self.alpha = alpha
        self.beta = beta
        self.total_run_time = total_run_time
        self.time_step = time_step
        self.max_iter = max_iter
        self.tol= tol 
        self.min_beta = min_beta
        self.max_beta = max_beta
        self.verbose = verbose
        self.seed = seed 
        
        
        
    def fit(self, events):
            
        beta_candidates = np.linspace(0, 30, 100)
        scores = []
        hawkes_models = {}
        for i, beta in enumerate(beta_candidates):
            hawkes_learner = HawkesExpKern(beta, 
                                           verbose=self.verbose, 
                                           max_iter=self.max_iter,
                                           tol=self.tol)
            hawkes_learner.fit(timestamps)
            hawkes_models[i] = hawkes_learner
            scores.append(hawkes_learner.score() )
    
        beta_scores_df = pd.DataFrame({'score': scores, 
                               'decay':beta_candidates}).sort_values(by = 'score', ascending = False)
        best_hawkes_model_index = beta_scores_df.index[0]
        best_hawkes_model = hawkes_models[best_hawkes_model_index]
        self.baseline = best_hawkes_model.baseline[0]
        self.alpha = best_hawkes_model.adjacency[0][0]
        self.beta = best_hawkes_model.decays
        
        
            
    
    def simulate(self)-> Tuple[np.ndarray, np.ndarray, np.ndarray]:
         
        hawkes = SimuHawkes(n_nodes=self.number_of_nodes, 
                            end_time=self.total_run_time, 
                            verbose=self.verbose, 
                            seed=self.seed)
        kernel = HawkesKernelExp(intensity=self.alpha, 
                                    decay= self.beta)
        hawkes.set_kernel(0, 0, kernel)
        hawkes.set_baseline(0, self.baseline)
        hawkes.track_intensity(self.time_step)
        hawkes.simulate()
        
        return hawkes.timestamps, hawkes.tracked_intensity, hawkes.intensity_tracked_times
        
        
    
    
    
    def get_conditional_intensity(self,
                                  events: np.ndarray, 
                                  current_step: float) -> float:

        return self.baseline + np.sum(self.alpha*self.beta * np.exp(-self.beta * (current_step - events)))            
    
    
    def get_next_arrival_time(self,
                                  events: np.ndarray, 
                                  current_step: float)-> float:
        
        predicted_intensity = self.get_conditional_intensity(events, current_step)
        
        return current_step + np.random.exponential(1 / predicted_intensity)
    